In [ ]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback


In [ ]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


In [ ]:
def create_index(tname):
    c = "CREATE INDEX IF NOT EXISTS wdpa_%s_intersections_idx ON wdpa_%s_intersections (wdpa_gid, code);" % (tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

def create_counts_index():
    c = "CREATE INDEX IF NOT EXISTS wdpa_raw_counts_idx ON wdpa_raw_counts (wdpa_gid);" 
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    
    
def vacuum_table(tname):
    c = "VACUUM ANALYZE wdpa_%s_intersections;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   

def vacuum_counts_table():
    c = "VACUUM ANALYZE wdpa_raw_counts;" 
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   
    

def select_into(tname):
    c = """
    SELECT 
      wdpa_gid, 
       sum(case when (
         code = 'CR' or 
         code = 'EN' or 
         code = 'VU' or 
         code = 'NT' or
         code = 'LC' 
       ) then 1 else 0 end) as all_count,      
       sum(case when code = 'CR' then 1 else 0 end) as cr_count,
       sum(case when code = 'CR' then intersection_area/species_area else 0 end) as cr_areas,
       sum(case when code = 'EN' then 1 else 0 end) as en_count,
       sum(case when code = 'EN' then intersection_area/species_area else 0 end) as en_areas,
       sum(case when code = 'VU' then 1 else 0 end) as vu_count,
       sum(case when code = 'VU' then intersection_area/species_area else 0 end) as vu_areas,   
       sum(case when code = 'NT' then 1 else 0 end) as nt_count,
       sum(case when code = 'NT' then intersection_area/species_area else 0 end) as nt_areas,   
       sum(case when code = 'LC' then 1 else 0 end) as lc_count,
       sum(case when code = 'LC' then intersection_area/species_area else 0 end) as lc_areas   
    INTO wdpa_raw_counts
    FROM wdpa_%s_intersections
    GROUP BY wdpa_gid 
    ORDER BY wdpa_gid
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd

def insert_into(tname):
    c = """
    INSERT INTO wdpa_raw_counts
    SELECT 
      wdpa_gid, 
       sum(case when (
         code = 'CR' or 
         code = 'EN' or 
         code = 'VU' or 
         code = 'NT' or
         code = 'LC' 
       ) then 1 else 0 end) as all_count,      
       sum(case when code = 'CR' then 1 else 0 end) as cr_count,
       sum(case when code = 'CR' then intersection_area/species_area else 0 end) as cr_areas,
       sum(case when code = 'EN' then 1 else 0 end) as en_count,
       sum(case when code = 'EN' then intersection_area/species_area else 0 end) as en_areas,
       sum(case when code = 'VU' then 1 else 0 end) as vu_count,
       sum(case when code = 'VU' then intersection_area/species_area else 0 end) as vu_areas,   
       sum(case when code = 'NT' then 1 else 0 end) as nt_count,
       sum(case when code = 'NT' then intersection_area/species_area else 0 end) as nt_areas,   
       sum(case when code = 'LC' then 1 else 0 end) as lc_count,
       sum(case when code = 'LC' then intersection_area/species_area else 0 end) as lc_areas   
    FROM wdpa_%s_intersections
    GROUP BY wdpa_gid 
    ORDER BY wdpa_gid
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd
    
    
def select_counts(tname, code):
    c = """
    SELECT 
      wdpa_gid, 
      count(*) as %s_count,
      sum(intersection_area/species_area)
    FROM wdpa_%s_intersections
    WHERE code = '%s'
    GROUP BY wdpa_gid 
    ORDER BY wdpa_gid
    
    LIMIT 10
    ;
    """
    c = c % (code, tname, code)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd       

    
def select_counts2(tname):
    c = """
    EXPLAIN ANALYZE
    SELECT 
      wdpa_gid, 
       sum(case when (
         code = 'CR' or 
         code = 'EN' or 
         code = 'VU' or 
         code = 'NT' or
         code = 'LC' 
       ) then 1 else 0 end) as all_count,      
       sum(case when code = 'CR' then 1 else 0 end) as cr_count,
       sum(case when code = 'CR' then intersection_area/species_area else 0 end) as cr_areas,
       sum(case when code = 'EN' then 1 else 0 end) as en_count,
       sum(case when code = 'EN' then intersection_area/species_area else 0 end) as en_areas,
       sum(case when code = 'VU' then 1 else 0 end) as vu_count,
       sum(case when code = 'VU' then intersection_area/species_area else 0 end) as vu_areas,   
       sum(case when code = 'NT' then 1 else 0 end) as nt_count,
       sum(case when code = 'NT' then intersection_area/species_area else 0 end) as nt_areas,   
       sum(case when code = 'LC' then 1 else 0 end) as lc_count,
       sum(case when code = 'LC' then intersection_area/species_area else 0 end) as lc_areas   
    FROM wdpa_%s_intersections
    GROUP BY wdpa_gid 
    ORDER BY wdpa_gid
    ;
    """
    c = c % (tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd       

    

In [ ]:
select_counts('mammals', 'CR')

In [ ]:
select_counts('mammals', 'VU')

In [ ]:
select_counts('mammals', 'EN')

In [ ]:
select_counts('mammals', 'NT')

In [ ]:
select_counts('mammals', 'LC')

In [ ]:
select_counts2('mammals')

In [ ]:
create_index('mammals')

In [ ]:
vacuum_table('mammals')

In [ ]:
select_counts2('mammals')

In [ ]:
tnames = []
for f in glob.glob("../../data/redlist/*/*.shp"):
    name = os.path.basename(f).split(".")[0]
    tnames.append(name.lower())
tnames.append('botw')
tnames.sort()

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(create_index, tname)
exe.shutdown()
None

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames:
    exe.submit(vacuum_table, tname)
exe.shutdown()
None

In [ ]:
select_into('amphibians')

In [ ]:
len(tnames)

In [ ]:
tnames[32]

In [ ]:
exe = SimpleProcessPoolExecutor(max_workers=8)
for tname in tnames[1:33]:
    exe.submit(insert_into, tname)
exe.shutdown()
None

In [ ]:
create_counts_index()

In [ ]:
vacuum_counts_table()